# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#)

Import melt inclusion and olivine data:

In [2]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures:

In [3]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [4]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the *PEC_MC_parameters* class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to do so when initialising the *PEC_MC_parameters* object

In this example we will use all error propagation options, which means we need to provide melt and olivine composition errors. We import these from .csv files containing error data for individual inclusions and olivines. This is just an example with made up errors, normally you would for example use errors from counting statistics.

In [5]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make very sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [6]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [7]:
melt_errors.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
PI032-04-01,1.02,0.64,0.14,0.47,0.69,0.10,0.08,0.05,0.18,0.05,0.05,0.17,0.21,0.22,0.10,0.12
PI032-04-02,1.06,0.84,0.29,0.46,0.54,0.05,0.03,0.01,0.06,0.18,0.04,0.24,0.12,0.13,0.12,0.04
PI041-02-02,1.04,0.90,0.09,0.55,0.49,0.22,0.21,0.01,0.16,0.04,0.00,0.15,0.23,0.04,0.17,0.06
PI041-03-01,0.98,0.64,0.10,0.40,0.53,0.12,0.12,0.08,0.01,0.03,0.00,0.05,0.36,0.06,0.14,0.04
PI041-03-03,1.02,0.54,0.31,0.42,0.68,0.18,0.03,0.02,0.11,0.06,0.10,0.18,0.08,0.13,0.06,0.15


In [8]:
olivine_errors.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO
PI032-04-01,1.06,0.60,1.22,0.01,0.09,0.08,0.02
PI032-04-02,0.85,0.57,1.28,0.14,0.07,0.24,0.06
PI041-02-02,1.11,0.44,1.30,0.08,0.10,0.01,0.19
PI041-03-01,0.99,0.50,1.31,0.14,0.16,0.15,0.01
PI041-03-03,0.95,0.64,1.21,0.13,0.11,0.10,0.15


Together with the *FeOi_prediction* object, we pass these as arguments to the *PEC_MC_parameters* object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors.

In [9]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True)

Now we can create the Monte Carlo model with the *PEC_MC* object

In [10]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

And finally run the Monte Carlo simulation with *n* iterations

In [11]:
pec_mc_model.run(n=30)

Monte Carlo loop
030/030
Equilibrating ... |██████████████████████████████| 100% [10/10] in 3.8s 
Correcting    ... |██████████████████████████████| 100% [10/10] in 7.7s 


Results are stored internally in the following attributes:

- **pec**: pandas DataFrame

    Average PEC extents (%) of the MC model and their one standard deviation errors.

- **inclusions_corr**: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions (wt. %)

- **inclusions_stddev**: pandas DataFrame

    One standard deviation errors on inclusions_corr (wt. %)

- **pec_MC**: pandas DataFrame

    PEC extents for individual iterations

- **inclusions_MC**: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [12]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [13]:
pec

,pec,stddev
name,,
PI032-04-01,7.610306,3.316469
PI032-04-02,8.439901,2.272792
PI041-02-02,-0.038287,1.872684
PI041-03-01,11.842898,2.427232
PI041-03-03,11.917466,2.549507
PI041-05-04,-4.179725,1.528762
PI041-05-06,1.985307,2.112468
PI041-07-01,10.164827,2.446106
PI041-07-02,10.36114,2.121078


In [14]:
inclusions_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,49.225505,14.172159,6.809178,9.858682,10.339014,3.657074,0.684866,0.11958,2.50246,0.270767,0.0,0.621056,1.409235,0.130014,0.128989,0.071422
PI032-04-02,48.373202,15.014667,6.681288,9.71435,10.342628,3.526555,0.928927,0.140009,2.619788,0.359857,0.0,0.635896,1.335552,0.105268,0.171611,0.050402
PI041-02-02,49.131,16.888683,4.78726,9.165667,10.209953,3.852465,1.060833,0.148752,2.80731,0.565634,0.0,0.485923,0.694583,0.050577,0.107273,0.044085
PI041-03-01,46.024306,15.761643,6.720101,11.01233,10.715729,3.385747,1.136367,0.124731,3.176457,0.558593,0.0,0.813721,0.334438,0.08947,0.087435,0.058933
PI041-03-03,45.404504,16.1004,6.608711,11.166312,10.781892,3.442238,1.172939,0.091262,3.293626,0.516705,0.0,0.826784,0.319254,0.10744,0.083379,0.084553
PI041-05-04,47.698818,18.869892,3.710478,9.475324,9.266708,4.645799,1.639656,0.139529,2.505798,0.824577,0.0,0.477799,0.483761,0.083575,0.118517,0.05977
PI041-05-06,46.481757,16.94855,4.6668,8.897771,11.488624,3.985123,1.422397,0.279944,3.649083,0.601789,0.0,0.648572,0.636959,0.1114,0.125803,0.05543
PI041-07-01,45.68204,15.570399,6.636411,9.845195,11.626865,3.16223,1.292888,0.142269,3.617378,0.565584,0.0,0.474943,1.032544,0.068765,0.168562,0.113928
PI041-07-02,45.826748,15.772595,6.575873,10.083668,11.488074,3.226758,1.356637,0.125242,3.587596,0.624291,0.0,0.391803,0.661864,0.074334,0.140502,0.064014


In [15]:
inclusions_errors

,SiO2_stddev,Al2O3_stddev,MgO_stddev,CaO_stddev,FeO_stddev,Na2O_stddev,K2O_stddev,MnO_stddev,TiO2_stddev,P2O5_stddev,Cr2O3_stddev,CO2_stddev,H2O_stddev,F_stddev,S_stddev,Cl_stddev
name,,,,,,,,,,,,,,,,
PI032-04-01,0.59422,0.762308,0.653379,0.319981,0.415913,0.143221,0.090486,0.044753,0.172232,0.050663,0.0,0.168754,0.222595,0.127023,0.087533,0.078746
PI032-04-02,0.669426,0.736676,0.487085,0.5383,0.262237,0.118603,0.031961,0.009114,0.06466,0.141731,0.0,0.19722,0.121502,0.095636,0.107109,0.040184
PI041-02-02,0.727487,0.904295,0.439686,0.714685,0.438036,0.201699,0.209324,0.012503,0.136585,0.039907,0.0,0.152079,0.203766,0.042266,0.109279,0.046554
PI041-03-01,0.458999,0.648953,0.48973,0.371997,0.153711,0.148447,0.097909,0.071275,0.062712,0.032468,0.0,0.044116,0.30517,0.052067,0.081452,0.032694
PI041-03-03,0.572917,0.550953,0.548193,0.455425,0.214591,0.163508,0.031852,0.021873,0.111447,0.059652,0.0,0.161665,0.080173,0.08431,0.054569,0.082757
PI041-05-04,0.73024,0.780735,0.338272,0.348367,0.269823,0.213203,0.141186,0.110291,0.080387,0.030506,0.0,0.205442,0.177641,0.103281,0.010116,0.010011
PI041-05-06,0.836939,0.888181,0.410997,0.406009,0.425431,0.30617,0.225649,0.179394,0.157371,0.044586,0.0,0.188608,0.260859,0.098389,0.041939,0.028109
PI041-07-01,0.684247,1.076029,0.605422,0.375308,0.308697,0.093279,0.034092,0.062611,0.059285,0.018462,0.0,0.042014,0.128164,0.090818,0.032907,0.093916
PI041-07-02,0.625162,0.655559,0.535515,0.345122,0.214292,0.184522,0.19359,0.044345,0.110064,0.01706,0.0,0.197698,0.168442,0.001738,0.050465,0.057982


In [16]:
pec_mc.head()

name,PI032-04-01,PI032-04-02,PI041-02-02,PI041-03-01,PI041-03-03,PI041-05-04,PI041-05-06,PI041-07-01,PI041-07-02,PI052-01-02
iteration,,,,,,,,,,
0,7.694471,7.225044,-0.833346,9.417683,11.851789,-2.557925,0.946222,7.726836,8.236837,-10.099346
1,2.446557,5.434909,-3.024565,8.744716,10.016657,-7.299171,3.464332,7.225332,8.836554,-11.14897
2,5.220569,7.652768,-0.007806,12.230761,15.255063,-2.278274,2.646015,14.967894,14.373463,-5.497467
3,4.124951,10.6711,-2.358204,9.166351,9.717959,-5.137773,-2.742601,6.480065,6.24303,-10.55384
4,7.531589,6.305011,-0.6,12.226346,13.138241,-4.753707,5.121282,10.835947,8.990396,-8.307569


The dataframes in *inclusions_MC* also have *isothermal_equilibration* and *Kd_equilibration* columns. These columns show if equilibration during stage [1] and [2] respectively was successful. In extreme cases, error propagation of melt and olivine compositions by random sampling can result in melt-olivine pairs that cannot be equilibrated without crystallising more than the entire inclusion or exchanging more Mg or Fe than is present in the inclusion. If that is the case, no corrected compositions are calculated and eiher the isothermal- of Kd-equilibration column is set to *False*.

In [17]:
inclusions_MC["PI032-04-01"].head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,isothermal_equilibration,Kd_equilibration
iteration,,,,,,,,,,,,,,,,,,
0,49.402757,13.208771,6.524614,9.679897,10.943335,3.746443,0.816437,0.120324,2.712037,0.299163,0.0,0.570752,1.751545,0.06995,0.0,0.153974,True,True
1,50.561645,14.859475,5.581907,9.601679,9.845601,3.746189,0.658142,0.047897,2.402562,0.351512,0.0,0.603629,1.616986,0.0,0.122776,0.0,True,True
2,49.525801,14.564647,6.978343,9.855032,9.82071,3.621338,0.640479,0.124194,2.299168,0.246702,0.0,0.512411,1.185164,0.375629,0.250383,0.0,True,True
3,49.412802,13.988131,5.782243,10.008637,10.476854,3.773997,0.802673,0.121538,2.408286,0.211379,0.0,0.785328,1.55905,0.259952,0.409129,0.0,True,True
4,49.53124,14.444628,6.826007,9.43851,10.144154,3.709044,0.844835,0.128246,2.46245,0.258969,0.0,0.649925,1.355617,0.124216,0.082159,0.0,True,True
